## Record_audio

In [5]:
import pyaudio
import wave

def record_audio(filename="output2.wav", duration=5, rate=16000, channels=1, chunk=1024, format=pyaudio.paInt16):
    audio = pyaudio.PyAudio()
    
    stream = audio.open(format=format, channels=channels,
                        rate=rate, input=True,
                        frames_per_buffer=chunk)
    
    print("Recording...")
    frames = []
    
    for _ in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)
    
    print("Recording finished.")
    
    stream.stop_stream()
    stream.close()
    audio.terminate()
    
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(channels)  # Chỉ 1 kênh (mono)
        wf.setsampwidth(audio.get_sample_size(format))
        wf.setframerate(rate)  # Tần số lấy mẫu 16kHz
        wf.writeframes(b''.join(frames))

In [6]:
record_audio()

Recording...
Recording finished.


# Using whisper-small

In [ ]:
import torch
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Load model and processor
model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
whisper = WhisperForConditionalGeneration.from_pretrained(model_name)

# Ensure model runs on GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}")
whisper.to(device)

## Load and transcribe

In [72]:
import torch
import torchaudio

# Load and preprocess audio
def load_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)
    return waveform.squeeze(0), 16000  # Convert to 1D tensor

# Transcribe function
def audio_to_text(audio_path):
    audio, sr = load_audio(audio_path)
    input_features = processor(audio, sampling_rate=sr, return_tensors="pt").input_features.to(device)

    # Generate transcription
    with torch.no_grad():
        predicted_ids = whisper.generate(input_features)
    
    return processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]


In [53]:
audio_to_text("output2.wav")

' Xin chào, tôi là Hùng, tôi đến từ Việt Nam, rất vui chào các bạn.'

## Translate using MarianMTModel

In [55]:
from transformers import MarianMTModel, MarianTokenizer

viToEnModel = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-vi-en")
viToEntokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-vi-en")
enToViModel = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-vi")
enToVitokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-vi")


def translate_vi_to_en(text):
    inputs = viToEntokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = viToEnModel.generate(**inputs)
    return viToEntokenizer.decode(translated[0], skip_special_tokens=True)


def translate_en_to_vi(text):
    inputs = enToVitokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = viToEnModel.generate(**inputs)
    return enToVitokenizer.decode(translated[0], skip_special_tokens=True)


c:\Users\Hokta\anaconda3\envs\GPU\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [56]:
translate_vi_to_en("Công đoạn đó nó khá là khó và bọn tao cũng đã cố gắng nhưng kết quả cũng chưa được tốt cho lắm")

"It was hard, and we tried, but it didn't work out so well."

In [57]:
translate_en_to_vi("This article is about the concept of residence. For the structure, see House. For other uses, see Home (disambiguation). 'Homes' redirects here. For other uses, see Homes (disambiguation).")

"mặt sách Nhân Alex according is Át the concept vậy 1/2., né Home I most?disambiguation). near names Sus' ♫s cái."

# Engine for speech

In [64]:
import pyttsx3

def text_to_speech(text):
    engine = pyttsx3.init()  # Khởi tạo engine
    engine.say(text)  # Đọc văn bản
    engine.runAndWait()  # Chạy lệnh đọc

text_to_speech("C4AI Aya Vision is an open weights research release of multimodal models with advanced capabilities optimized for real-time inference on edge devices.")



## Combine

In [68]:
# Combine all functions
def transcribe_translate_speak(file_path):
    text = audio_to_text(file_path)
    translated_text = translate_vi_to_en(text)
    text_to_speech(translated_text)

In [69]:
transcribe_translate_speak("output2.wav")